# Day 5

In [1]:
import $ivy.`org.scalaz::scalaz-core:7.2.22`
import $ivy.`org.scalaz::scalaz-effect:7.2.22`
import $ivy.`org.scalaz::scalaz-typelevel:7.1.17`
import $ivy.`org.scalaz::scalaz-scalacheck-binding:7.2.21`
import scalaz._
import Scalaz._
import scalacheck.ScalazProperties._
import scalacheck.ScalazArbitrary._
import scalacheck.ScalaCheckBinding._

import $ivy.$                               

import $ivy.$                                 

import $ivy.$                                    

import $ivy.$                                             

import scalaz._

import Scalaz._

import scalacheck.ScalazProperties._

import scalacheck.ScalazArbitrary._

import scalacheck.ScalaCheckBinding._

## モナドがいっぱい

```scala
trait Monad[F[_]] extends Applicative[F] with Bind[F] { self =>
  ////
}
```

## Bind

```scala
trait Bind[F[_]] extends Apply[F] { self =>
  /** Equivalent to `join(map(fa)(f))`. */
  def bind[A, B](fa: F[A])(f: A => F[B]): F[B]
}
```

```scala
/** Wraps a value `self` and provides methods related to `Bind` */
trait BindOps[F[_],A] extends Ops[F[A]] {
  implicit def F: Bind[F]
  ////
  import Liskov.<~<

  def flatMap[B](f: A => F[B]) = F.bind(self)(f)
  def >>=[B](f: A => F[B]) = F.bind(self)(f)
  def ∗[B](f: A => F[B]) = F.bind(self)(f)
  def join[B](implicit ev: A <~< F[B]): F[B] = F.bind(self)(ev(_))
  def μ[B](implicit ev: A <~< F[B]): F[B] = F.bind(self)(ev(_))
  def >>[B](b: F[B]): F[B] = F.bind(self)(_ => b)
  def ifM[B](ifTrue: => F[B], ifFalse: => F[B])(implicit ev: A <~< Boolean): F[B] = {
    val value: F[Boolean] = Liskov.co[F, A, Boolean](ev)(self)
    F.ifM(value, ifTrue, ifFalse)
  }
  ////
}
```

In [2]:
3.some flatMap { x => (x + 1).some }

res1: Option[Int] = Some(4)

In [3]:
(none: Option[Int]) flatMap { x => (x + 1).some }

res2: Option[Int] = None

## Monad

In [4]:
Monad[Option].point("WHAT")

res3: Option[String] = Some("WHAT")

In [5]:
9.some flatMap { x => Monad[Option].point(x * 10) }

res4: Option[Int] = Some(90)

In [6]:
(none: Option[Int]) flatMap { x => Monad[Option].point(x * 10) }

res5: Option[Int] = None

## 綱渡り

In [7]:
type Birds = Int

defined type Birds

In [8]:
case class Pole(left: Birds, right: Birds)

defined class Pole

In [9]:
case class Pole(left: Birds, right: Birds) {
    def landLeft(n: Birds): Pole = copy(left = left + n)
    def landRight(n: Birds): Pole = copy(right = right + n) 
}

defined class Pole

In [10]:
Pole(0, 0).landLeft(2)

res9: Pole = Pole(2, 0)

In [11]:
Pole(1, 2).landRight(1)

res10: Pole = Pole(1, 3)

In [12]:
Pole(1, 2).landRight(-1)

res11: Pole = Pole(1, 1)

In [13]:
Pole(0, 0).landLeft(1).landRight(1).landLeft(2)

res12: Pole = Pole(3, 1)

In [14]:
Pole(0, 0).landLeft(1).landRight(4).landLeft(-1).landRight(-2)

res13: Pole = Pole(0, 2)

In [15]:
case class Pole(left: Birds, right: Birds) {
    def landLeft(n: Birds): Option[Pole] = 
        if (math.abs((left + n) - right) < 4) copy(left = left + n).some
        else none
    def landRight(n: Birds): Option[Pole] =
        if (math.abs(left - (right + n)) < 4) copy(right = right + n).some
        else none
}

defined class Pole

In [16]:
Pole(0, 0).landLeft(2)

res15: Option[Pole] = Some(Pole(2, 0))

In [17]:
Pole(0, 3).landLeft(10)

res16: Option[Pole] = None

In [18]:
Pole(0, 0).landRight(1) flatMap {_.landLeft(2)}

res17: Option[Pole] = Some(Pole(2, 1))

In [19]:
(none: Option[Pole]) flatMap {_.landLeft(2)}

res18: Option[Pole] = None

In [20]:
Monad[Option].point(Pole(0, 0)) flatMap {_.landRight(2)} flatMap {_.landLeft(2)} flatMap {_.landRight(2)}

res19: Option[Pole] = Some(Pole(2, 4))

In [21]:
Monad[Option].point(Pole(0, 0)) >>= {_.landRight(2)} >>= {_.landLeft(2)} >>= {_.landRight(2)}

res20: Option[Pole] = Some(Pole(2, 4))

In [22]:
Monad[Option].point(Pole(0, 0)) >>= {_.landLeft(1)} >>= {_.landRight(4)} >>= {_.landLeft(-1)} >>= {_.landRight(-2)}

res21: Option[Pole] = None

## ロープの上のバナナ

In [23]:
case class Pole(left: Birds, right: Birds) {
    def landLeft(n: Birds): Option[Pole] = 
        if (math.abs((left + n) - right) < 4) copy(left = left + n).some
        else none
    def landRight(n: Birds): Option[Pole] =
        if (math.abs(left - (right + n)) < 4) copy(right = right + n).some
        else none
    def banana: Option[Pole] = none
}

defined class Pole

In [24]:
Monad[Option].point(Pole(0, 0)) >>= {_.landLeft(1)} >>= {_.banana} >>= {_.landRight(1)}

res23: Option[Pole] = None

In [25]:
(none: Option[Int]) >> 3.some

res24: Option[Int] = None

In [26]:
3.some >> 4.some

res25: Option[Int] = Some(4)

In [27]:
3.some >> (none: Option[Int])

res26: Option[Int] = None

In [27]:
Monad[Option].point(Pole(0, 0)) >>= {_.landLeft(1)} >> (none: Option[Pole]) >>= {_.landRight(1)}

cmd27.sc:1: missing parameter type for expanded function ((x$1: <error>) => x$1.landLeft(1))
val res27 = Monad[Option].point(Pole(0, 0)) >>= {_.landLeft(1)} >> (none: Option[Pole]) >>= {_.landRight(1)}
                                                 ^

: 

In [28]:
Monad[Option].point(Pole(0, 0)).>>=({_.landLeft(1)}).>>(none: Option[Pole]).>>=({_.landRight(1)})

res27: Option[Pole] = None

In [29]:
(Monad[Option].point(Pole(0, 0)) >>= {_.landLeft(1)}) >> (none: Option[Pole]) >>= {_.landRight(1)}

res28: Option[Pole] = None

## for 構文

In [30]:
3.some >>= { x => "!".some >>= { y => (x.shows + y).some } }

res29: Option[String] = Some("3!")

In [31]:
3.some >>= { x => "!".some >>= { y => (x.shows + y).some } }

res30: Option[String] = Some("3!")

In [32]:
(none: Option[Int]) >>= { x => "!".some >>= { y => (x.shows + y).some } }

res31: Option[String] = None

In [33]:
3.some >>= { x => "!".some >>= { y => (none: Option[String]) } }

res32: Option[String] = None

In [34]:
for {
    x <- 3.some
    y <- "!".some
} yield (x.shows + y)

res33: Option[String] = Some("3!")

## 帰ってきたピエール

In [35]:
def routine: Option[Pole] =
    for {
        start <- Monad[Option].point(Pole(0, 0))
        first <- start.landLeft(2)
        second <- first.landRight(2)
        third <- second.landLeft(1)
    } yield third

defined function routine

In [36]:
routine

res35: Option[Pole] = Some(Pole(3, 2))

In [37]:
def routine: Option[Pole] =
    for {
        start <- Monad[Option].point(Pole(0, 0))
        first <- start.landLeft(2)
        _ <- (none: Option[Pole])
        second <- first.landRight(2)
        third <- second.landLeft(1)
    } yield third

defined function routine

In [38]:
routine

res37: Option[Pole] = None

## パターンマッチングと失敗

In [39]:
def justH: Option[Char] =
    for {
        (x :: xs) <- "hello".toList.some
    } yield x

defined function justH

In [40]:
justH

res39: Option[Char] = Some('h')

In [41]:
def wopwop: Option[Char] =
    for {
        (x :: xs) <- "".toList.some
    } yield x

defined function wopwop

In [42]:
wopwop

res41: Option[Char] = None

## List モナド

In [43]:
^(List(1, 2, 3), List(10, 100, 100)) {_ * _}

res42: List[Int] = List(10, 100, 100, 20, 200, 200, 30, 300, 300)

In [44]:
List(3, 4, 5) >>= {x => List(x, -x)}

res43: List[Int] = List(3, -3, 4, -4, 5, -5)

In [45]:
for {
    n <- List(1, 2)
    ch <- List('a', 'b')
} yield (n, ch)

res44: List[(Int, Char)] = List((1, 'a'), (1, 'b'), (2, 'a'), (2, 'b'))

## MonadPlus と guard 関数

In [46]:
for {
    x <- 1 |-> 50 if x.shows contains '7'
} yield x

res45: List[Int] = List(7, 17, 27, 37, 47)

## Plus、PlusEmpty、と ApplicativePlus

In [47]:
List(1, 2, 3) <+> List(4, 5, 6)

res46: List[Int] = List(1, 2, 3, 4, 5, 6)

## MonadPlus 再び

In [48]:
(1 |-> 50) filter { x => x.shows contains '7' }

res47: List[Int] = List(7, 17, 27, 37, 47)

## 騎士の旅

In [49]:
case class KnightPos(c: Int, r: Int)

defined class KnightPos

In [50]:
case class KnightPos(c: Int, r: Int) {
    def move: List[KnightPos] =
        for {
            KnightPos(c2, r2) <- List(KnightPos(c + 2, r - 1), KnightPos(c + 2, r + 1),
                KnightPos(c - 2, r - 1), KnightPos(c - 2, r + 1),
                KnightPos(c + 1, r - 2), KnightPos(c + 1, r + 2),
                KnightPos(c - 1, r - 2), KnightPos(c - 1, r + 2)) if (
                ((1 |-> 8) contains c2) && ((1 |-> 8) contains r2))
        } yield KnightPos(c2, r2)
    }

defined class KnightPos

In [51]:
KnightPos(6, 2).move

res50: List[KnightPos] = List(
  KnightPos(8, 1),
  KnightPos(8, 3),
  KnightPos(4, 1),
  KnightPos(4, 3),
  KnightPos(7, 4),
  KnightPos(5, 4)
)

In [52]:
KnightPos(8, 1).move

res51: List[KnightPos] = List(KnightPos(6, 2), KnightPos(7, 3))

In [53]:
case class KnightPos(c: Int, r: Int) {
    def move: List[KnightPos] =
        for {
            KnightPos(c2, r2) <- List(KnightPos(c + 2, r - 1), KnightPos(c + 2, r + 1),
            KnightPos(c - 2, r - 1), KnightPos(c - 2, r + 1),
            KnightPos(c + 1, r - 2), KnightPos(c + 1, r + 2),
            KnightPos(c - 1, r - 2), KnightPos(c - 1, r + 2)) if (
            ((1 |-> 8) element c2) && ((1 |-> 8) contains r2))
        } yield KnightPos(c2, r2)
    def in3: List[KnightPos] =
        for {
            first <- move
            second <- first.move
            third <- second.move
        } yield third
    def canReachIn3(end: KnightPos): Boolean = in3 contains end
}

defined class KnightPos

In [54]:
KnightPos(6, 2) canReachIn3 KnightPos(6, 1)

res53: Boolean = true

In [55]:
KnightPos(6, 2) canReachIn3 KnightPos(7, 3)

res54: Boolean = false

## Monad則

### 左単位元

In [55]:
(Monad[Option].point(3) >>= { x => (x + 100000).some }) assert_=== 3 |> { x => (x + 100000).some }

cmd55.sc:1: type mismatch;
 found   : Int(3)
 required: ?{def |>: ?}
Note that implicit conversions are not applicable because they are ambiguous:
 both method Pipeable in trait Extensions of type [T](t: T)ammonite.ops.Pipeable[T]
 and method ToIdOps in trait ToIdOps of type [A](a: A)scalaz.syntax.IdOps[A]
 are possible conversion functions from Int(3) to ?{def |>: ?}
val res55 = (Monad[Option].point(3) >>= { x => (x + 100000).some }) assert_=== 3 |> { x => (x + 100000).some }
                                                                               ^

: 

### 右単位元

In [56]:
("move on up".some flatMap {Monad[Option].point(_)}) assert_=== "move on up".some

### 結合律

In [57]:
Monad[Option].point(Pole(0, 0)) >>= {_.landRight(2)} >>= {_.landLeft(2)} >>= {_.landRight(2)}

res56: Option[Pole] = Some(Pole(2, 4))

In [58]:
Monad[Option].point(Pole(0, 0)) >>= { x =>
    x.landRight(2) >>= { y =>
    y.landLeft(2) >>= { z =>
    z.landRight(2)
    }}}

res57: Option[Pole] = Some(Pole(2, 4))

In [59]:
monad.laws[Option].check

+ monad.applicative.apply.functor.invariantFunctor.identity: OK, passed 100
   tests.
+ monad.applicative.apply.functor.invariantFunctor.composite: OK, passed 10
  0 tests.
+ monad.applicative.apply.functor.identity: OK, passed 100 tests.
+ monad.applicative.apply.functor.composite: OK, passed 100 tests.
+ monad.applicative.apply.composition: OK, passed 100 tests.
+ monad.applicative.identity: OK, passed 100 tests.
+ monad.applicative.homomorphism: OK, passed 100 tests.
+ monad.applicative.interchange: OK, passed 100 tests.
+ monad.applicative.map consistent with ap: OK, passed 100 tests.
+ monad.bind.apply.functor.invariantFunctor.identity: OK, passed 100 tests.
+ monad.bind.apply.functor.invariantFunctor.composite: OK, passed 100 tests
  .
+ monad.bind.apply.functor.identity: OK, passed 100 tests.
+ monad.bind.apply.functor.composite: OK, passed 100 tests.
+ monad.bind.apply.composition: OK, passed 100 tests.
+ monad.bind.associativity: OK, passed 100 tests.
+ monad.bind.ap consisten